In [1]:
import pandas as pd
import numpy as np
import math
import random
import docx
from docx.shared import Cm

pd.set_option('display.max_rows', 500)

In [2]:
vf_rooms = ['Pre-K', 'Kindergarten', 'Math Lab', 'Math', '3rd Grade', '2nd Grade', '1st Grade']
ep_rooms = ['Art', 'Pri. Sci & Spanish', '6th Grade', '5-8 Science', '4th Grade']
cr_rooms = ['Pre-K', 'Kindergarten', 'Math Lab', 'Math', '3rd Grade', '2nd Grade', '1st Grade', 'Art']
oo_rooms = ['5th Grade', 'Library', '7th Grade', '8th Grade']

In [3]:
raw_speech = pd.read_csv("roster.csv")
names = list(raw_speech['Name'])
cold_reading_raw = list(raw_speech['Cold Reading'])
panel_raw = list(raw_speech['Panel Debate'])
vf_raw = list(raw_speech['Volte Face'])
ep_raw = list(raw_speech['Elevator Pitch'])
oo_raw = list(raw_speech['Original Oratory'])
cr_codes = {}
cr = []
vf_codes = {}
vf = []
ep_codes = {}
ep = []
oo_codes = {}
oo = []
for i in range(len(names)):
  if isinstance(cold_reading_raw[i], str):
    code = i + 1500
    if isinstance(panel_raw[i], str):
      code -= 500
    cr_codes[code] = names[i]
    cr.append(code)
  if isinstance(vf_raw[i], str):
    code = i + 2500
    if isinstance(oo_raw[i], str):
      code -= 500
    vf_codes[code] = names[i]
    vf.append(code)
  if isinstance(ep_raw[i], str):
    code = i + 3500
    if isinstance(oo_raw[i], str):
      code -= 500
    ep_codes[code] = names[i]
    ep.append(code)
  if isinstance(oo_raw[i], str):
    code = i + 4000
    if isinstance(ep_raw[i], str) and isinstance(vf_raw[i], str):
      code += 200
    if isinstance(ep_raw[i], str) or isinstance(vf_raw[i], str):
      code += 200
    oo_codes[code] = names[i]
    oo.append(code)

In [4]:
vf_records = {code : [0, 0] for code in vf}
ep_records = {code : [0, 0] for code in ep}
cr_records = {code : [0, 0] for code in cr}
oo_records = {code : [0, 0] for code in oo}

In [5]:
def create_round(entries, rooms):
  temp = [[] for _ in range(len(rooms))]
  random.shuffle(entries)
  for i in range(len(entries)):
    index = i % len(rooms)
    temp[index].append(entries[i])
  round = []
  for room in temp:
    round.append(sorted(room))
  return round

In [8]:
vf_round_1 = create_round(vf, vf_rooms)
ep_round_1 = create_round(ep, ep_rooms)
cr_round_1 = create_round(cr, cr_rooms)
oo_round_1 = create_round(oo, oo_rooms)

vf_round_2 = create_round(vf, vf_rooms)
ep_round_2 = create_round(ep, ep_rooms)
cr_round_2 = create_round(cr, cr_rooms)
oo_round_2 = create_round(oo, oo_rooms)

In [6]:
def write_master(template, pairings, code, rooms, event, round):
  for i in range(len(rooms)):
    save_str = event + str(round) + rooms[i] + "Master.docx"
    temp = docx.Document(template)
    temp.paragraphs[2].text +=  str(round)
    temp.paragraphs[4].text += rooms[i]
    table = temp.tables[0]
    for j in range(len(pairings[i])):
      id = pairings[i][j]
      table.cell(j+1, 0).text = str(code[id]) + " - " + str(id)
    temp.save(save_str)

In [12]:
write_master("vf_mb.docx", vf_round_1, vf_codes, vf_rooms, "VF", 1)
write_master("ep_mb.docx", ep_round_1, ep_codes, ep_rooms, "EP", 1)
write_master("cr_mb.docx", cr_round_1, cr_codes, cr_rooms, "CR", 1)
write_master("oo_mb.docx", oo_round_1, oo_codes, oo_rooms, "OO", 1)

In [23]:
write_master("vf_mb.docx", vf_round_2, vf_codes, vf_rooms, "VF", 2)
write_master("ep_mb.docx", ep_round_2, ep_codes, ep_rooms, "EP", 2)
write_master("cr_mb.docx", cr_round_2, cr_codes, cr_rooms, "CR", 2)
write_master("oo_mb.docx", oo_round_2, oo_codes, oo_rooms, "OO", 2)

In [7]:
def create_pairings(pairings, code, rooms, event, round, time):
  with open(event + 'Round' + str(round) + '.txt', 'w') as f:
    f.write(event + ' Round ' + str(round) + '\nStart Time: ' + time + '\nSpeeches in order that names are listed\n\n\n')
    for i in range(len(rooms)):
      f.write("Room: "+ str(rooms[i]) + '\n\n')
      for num in pairings[i]:
        f.write(str(code[num]) + '\n')
      f.write('\n')

In [22]:
create_pairings(vf_round_1, vf_codes, vf_rooms, "Volte Face", 1, '9:50 AM')
create_pairings(ep_round_1, ep_codes, ep_rooms, "Elevator Pitch", 1, '9:50 AM')
create_pairings(cr_round_1, cr_codes, cr_rooms, "Cold Reading", 1, '8:50 AM')
create_pairings(oo_round_1, oo_codes, oo_rooms, "Original Oratory", 1, '9:50 AM')

create_pairings(vf_round_2, vf_codes, vf_rooms, "Volte Face", 2, '1:00 PM')
create_pairings(ep_round_2, ep_codes, ep_rooms, "Elevator Pitch", 2, '1:00 PM')
create_pairings(cr_round_2, cr_codes, cr_rooms, "Cold Reading", 2, '10:40 AM')
create_pairings(oo_round_2, oo_codes, oo_rooms, "Original Oratory", 2, '1:00 PM')

In [36]:
def ingest_ballot():
  print("Input Event (vf, ep, cr, oo)")
  event = input()
  while event not in {'vf', 'ep', 'cr', 'oo'}:
    print('Wrong event input')
    event = input()
  print("Input codes, separated by spaces, in order from first to last")
  codes_raw = input()
  codes = list(codes_raw.split(" "))
  for i in range(len(codes)): codes[i] = int(codes[i])
  for i in range(len(codes)):
    place = i+1
    if event == 'vf':
      vf_records[codes[i]][0] += place
      vf_records[codes[i]][1] += 1
    elif event == 'ep':
      ep_records[codes[i]][0] += place
      ep_records[codes[i]][1] += 1
    elif event == 'cr':
      cr_records[codes[i]][0] += place
      cr_records[codes[i]][1] += 1
    elif event == 'oo':
      oo_records[codes[i]][0] += place
      oo_records[codes[i]][1] += 1
    else:
      print("Error")

In [8]:
def get_current_sich():
  combo = {}
  for key in vf_records:
    combo[key] = [vf_codes[key]] + vf_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()
  print(df)
  combo = {}
  for key in ep_records:
    combo[key] = [ep_codes[key]] + ep_records[key]
  df1 = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df1['Rank'] = df1['Places Sum'].rank()
  print(df1)
  combo = {}
  for key in cr_records:
    combo[key] = [cr_codes[key]] + cr_records[key]
  df2 = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df2['Rank'] = df2['Places Sum'].rank()
  print(df2)
  combo = {}
  for key in oo_records:
    combo[key] = [oo_codes[key]] + oo_records[key]
  df3 = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df3['Rank'] = df3['Places Sum'].rank()
  print(df3)

In [9]:
ingest_ballot()
get_current_sich()

                           Name  Places Sum  Ballots In  Rank
2000          Mary Lydia Cassie           0           0  35.0
2001                Anna Elsie            0           0  35.0
2503    Elizabeth Susan Melissa           0           0  35.0
2004         Minnie Mollie Lyda           0           0  35.0
2005             Margaret Alma            0           0  35.0
2006                 Ida Addie            0           0  35.0
2507             Alice Georgia            0           0  35.0
2508              Bertha Eliza            0           0  35.0
2009            Sarah Lulu Mary           0           0  35.0
2510              Annie Nannie            0           0  35.0
2512               Ella Amanda            0           0  35.0
2513            Florence Belle            0           0  35.0
2516                Laura Ruth            0           0  35.0
2517              Nellie Viola            0           0  35.0
2019        Carrie Amelia Annie           0           0  35.0
2520    

In [10]:
combo = {}
for key in cr_records:
  combo[key] = [cr_codes[key]] + cr_records[key]
  df2 = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df2['Rank'] = df2['Places Sum'].rank()
df2 = df2[df2['Rank'] < 40.5]
cr_quarters = list(df2.index.values)
print(cr_quarters)

[1500, 1002, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1013, 1014, 1015, 1016, 1017, 1519, 1521, 1522, 1023, 1524, 1525, 1527, 1528, 1529, 1032, 1535, 1536, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1547, 1549, 1550, 1051, 1552, 1053, 1055, 1056, 1557, 1559, 1560, 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569, 1571, 1572, 1574, 1076, 1578, 1579, 1080, 1581, 1582, 1583, 1084, 1089, 1091, 1592, 1096, 1600, 1601]


In [95]:
cr_quarters_rooms = ['Math Lab', 'Math', '3rd Grade', '2nd Grade']
cr_quarters_round = create_round(cr_quarters, cr_quarters_rooms)
write_master("cr_mb.docx", cr_quarters_round, cr_codes, cr_quarters_rooms, "CR", 3)
create_pairings(cr_quarters_round, cr_codes, cr_quarters_rooms, "Cold Reading", 3, '1:45 PM')

In [181]:
combo = {}
for key in ep_records:
  combo[key] = [ep_codes[key]] + ep_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()
df = df[df['Rank'] < 20.5]
ep_semis = list(df.index.values)

In [188]:
cr_quarters_records = {code : [0, 0] for code in cr_quarters}

In [201]:
codes_raw = input()
codes = list(codes_raw.split(" "))
for i in range(len(codes)): codes[i] = int(codes[i])
for i in range(len(codes)):
  place = i+1
  cr_quarters_records[codes[i]][0] += place
  cr_quarters_records[codes[i]][1] += 1

In [203]:
combo = {}
for key in cr_quarters_records:
  combo[key] = [cr_codes[key]] + cr_quarters_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()
df = df[df['Rank'] < 20.5]
cr_semis = list(df.index.values)
print(len(cr_semis))

20


In [204]:
cr_semis_rooms = ['Art', 'Pri. Sci & Spanish']
cr_semis_round = create_round(cr_semis, cr_semis_rooms)
write_master("cr_mb.docx", cr_semis_round, cr_codes, cr_semis_rooms, "CR", 4)
create_pairings(cr_semis_round, cr_codes, cr_semis_rooms, "Cold Reading", 4, '3:30 PM')

In [182]:
combo = {}
for key in vf_records:
  combo[key] = [vf_codes[key]] + vf_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()
df = df[df['Rank'] < 29.5]
vf_semis = list(df.index.values)
print(len(vf_semis))

27


In [183]:
combo = {}
for key in oo_records:
  combo[key] = [oo_codes[key]] + oo_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()
df = df[df['Rank'] < 16.5]
oo_semis = list(df.index.values)
print(len(oo_semis))

15


In [184]:
oo_semis_rooms = ['Math Lab', 'Math']
oo_semis_round = create_round(oo_semis, oo_semis_rooms)
write_master("oo_mb.docx", oo_semis_round, oo_codes, oo_semis_rooms, "OO", 3)
create_pairings(oo_semis_round, oo_codes, oo_semis_rooms, "Original Oratory", 3, '3:00 PM')

In [187]:
vf_semis_rooms = ['3rd Grade', '2nd Grade', '5-8 Science']
vf_semis_round = create_round(vf_semis, vf_semis_rooms)
write_master("vf_mb.docx", vf_semis_round, vf_codes, vf_semis_rooms, "VF", 3)
create_pairings(vf_semis_round, vf_codes, vf_semis_rooms, "Volte Face", 3, '3:00 PM')

In [186]:
ep_semis_rooms = ["5th Grade", "Library"]
ep_semis_round = create_round(ep_semis, ep_semis_rooms)
write_master("ep_mb.docx", ep_semis_round, ep_codes, ep_semis_rooms, "EP", 3)
create_pairings(ep_semis_round, ep_codes, ep_semis_rooms, "Elevator Pitch", 3, '3:00 PM')

In [ ]:
combo = {}
for key in cr_quarters_records:
  combo[key] = [cr_codes[key]] + cr_quarters_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()

df = df[df['Rank'] < 20.5]
cr_semis = list(df.index.values)
print(len(cr_semis))

In [ ]:
cr_semis_rooms = ['Art', 'Pri. Sci & Spanish']
cr_semis_round = create_round(cr_semis, cr_semis_rooms)
write_master("cr_mb.docx", cr_semis_round, cr_codes, cr_semis_rooms, "CR", 4)
create_pairings(cr_semis_round, cr_codes, cr_semis_rooms, "Cold Reading", 4, '3:30 PM')

In [206]:
cr_semis_records = {code : [0, 0] for code in cr_semis}
oo_semis_records = {code : [0, 0] for code in oo_semis}
vf_semis_records = {code : [0, 0] for code in vf_semis}
ep_semis_records = {code : [0, 0] for code in ep_semis}

COLD READING

In [250]:
codes_raw = input()
codes = list(codes_raw.split(" "))
for i in range(len(codes)): codes[i] = int(codes[i])
for i in range(len(codes)):
  place = i+1
  cr_semis_records[codes[i]][0] += place
  cr_semis_records[codes[i]][1] += 1

In [12]:
combo = {}
for key in cr_semis_records:
  combo[key] = [cr_codes[key]] + cr_semis_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()
print(len(df))
print(df)
df = df[df['Rank'] < 10.5]
cr_finals = list(df.index.values)
cr_finals = cr_finals[:7] + cr_finals[8:]
print(len(cr_finals))

'combo = {}\nfor key in cr_semis_records:\n  combo[key] = [cr_codes[key]] + cr_semis_records[key]\n  df = pd.DataFrame.from_dict(combo, orient=\'index\', columns=["Name", "Places Sum", "Ballots In"])\n  df[\'Rank\'] = df[\'Places Sum\'].rank()\nprint(len(df))\nprint(df)\ndf = df[df[\'Rank\'] < 10.5]\ncr_finals = list(df.index.values)\ncr_finals = cr_finals[:7] + cr_finals[8:]\nprint(len(cr_finals))'

In [254]:
cr_finals_rooms = ['Math Lab']
cr_finals_round = create_round(cr_finals, cr_finals_rooms)
write_master("cr_mb.docx", cr_finals_round, cr_codes, cr_finals_rooms, "CR", 5)
create_pairings(cr_finals_round, cr_codes, cr_finals_rooms, "Cold Reading", 5, '4:15 PM')

VOLTE FACE

In [236]:
codes_raw = input()
codes = list(codes_raw.split(" "))
for i in range(len(codes)): codes[i] = int(codes[i])
for i in range(len(codes)):
  place = i+1
  vf_semis_records[codes[i]][0] += place
  vf_semis_records[codes[i]][1] += 1

In [14]:
print(vf_semis_records)

In [16]:
combo = {}
for key in vf_semis_records:
  combo[key] = [vf_codes[key]] + vf_semis_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()
print(len(df))
print(df)
df = df[df['Rank'] < 10.5]
vf_finals = list(df.index.values)
vf_finals = [vf_finals[0]] + vf_finals[2:]
print(vf_finals)
print(len(vf_finals))

'combo = {}\nfor key in vf_semis_records:\n  combo[key] = [vf_codes[key]] + vf_semis_records[key]\n  df = pd.DataFrame.from_dict(combo, orient=\'index\', columns=["Name", "Places Sum", "Ballots In"])\n  df[\'Rank\'] = df[\'Places Sum\'].rank()\nprint(len(df))\nprint(df)\ndf = df[df[\'Rank\'] < 10.5]\nvf_finals = list(df.index.values)\nvf_finals = [vf_finals[0]] + vf_finals[2:]\nprint(vf_finals)\nprint(len(vf_finals))'

In [244]:
vf_finals_rooms = ['Math']
vf_finals_round = create_round(vf_finals, vf_finals_rooms)
write_master("vf_mb.docx", vf_finals_round, vf_codes, vf_finals_rooms, "VF", 4)
create_pairings(vf_finals_round, vf_codes, vf_finals_rooms, "Volte Face", 4, '4:15 PM')

ELEVATOR PITCH

In [230]:
codes_raw = input()
codes = list(codes_raw.split(" "))
for i in range(len(codes)): codes[i] = int(codes[i])
for i in range(len(codes)):
  place = i+1
  ep_semis_records[codes[i]][0] += place
  ep_semis_records[codes[i]][1] += 1

In [ ]:
combo = {}
for key in ep_semis_records:
  combo[key] = [ep_codes[key]] + ep_semis_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()
print(len(df))
print(df)
df = df[df['Rank'] < 10.5]
ep_finals = list(df.index.values)
print(len(ep_finals))

In [232]:
ep_finals_rooms = ['7th Grade']
ep_finals_round = create_round(ep_finals, ep_finals_rooms)
write_master("ep_mb.docx", ep_finals_round, ep_codes, ep_finals_rooms, "EP", 4)
create_pairings(ep_finals_round, ep_codes, ep_finals_rooms, "Elevator Pitch", 4, '4:15 PM')

ORIG ORAT

In [221]:
codes_raw = input()
codes = list(codes_raw.split(" "))
for i in range(len(codes)): codes[i] = int(codes[i])
for i in range(len(codes)):
  place = i+1
  oo_semis_records[codes[i]][0] += place
  oo_semis_records[codes[i]][1] += 1

In [ ]:
combo = {}
for key in oo_semis_records:
  combo[key] = [oo_codes[key]] + oo_semis_records[key]
  df = pd.DataFrame.from_dict(combo, orient='index', columns=["Name", "Places Sum", "Ballots In"])
  df['Rank'] = df['Places Sum'].rank()
print(len(df))
print(df)
df = df[df['Rank'] < 8.5]
oo_finals = list(df.index.values)
print(len(oo_finals))

In [225]:
oo_finals_rooms = ['6th Grade']
oo_finals_round = create_round(oo_finals, oo_finals_rooms)
write_master("oo_mb.docx", oo_finals_round, oo_codes, oo_finals_rooms, "OO", 4)
create_pairings(oo_finals_round, oo_codes, oo_finals_rooms, "Original Oratory", 4, '4:15 PM')